In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch_geometric
import pickle
import optuna
from torch_geometric.nn import GCNConv

### data

In [2]:
# model structure
class GCN(torch.nn.Module):
    def __init__(self, size1, size2, size3):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(1969, size1)
        self.conv2 = GCNConv(size1, size2)
        self.fc1 = torch.nn.Linear(size2, size3)
        self.fc2 = torch.nn.Linear(size3, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x1 = torch.relu(x)
        x = self.fc1(x1)
        x = torch.relu(x)
        x = self.fc2(x)       
        return x, x1

def train(model, data, optimizer, mask):
    model.train()
    optimizer.zero_grad()
    out, _ = model(data)
    loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out, _ = model(data)
        loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    return loss.item()

def objective(trial: optuna.Trial):   
    size1 = trial.suggest_int("size1", 32, 1024)
    size2 = trial.suggest_int("size2", 4, 512)
    size3 = trial.suggest_int("size3", 4, 512)
    num_epochs = trial.suggest_int("num_epochs", 1, 200)
    lr = trial.suggest_float("lr", 1e-5, 1e-2)

    model = GCN(size1, size2, size3).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # find best hyperparameter by training on the training set and get loss on the validating set
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, data, optimizer, data.train_mask)
        val_loss = evaluate(model, data, data.valid_mask)
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return val_loss

In [3]:
for rs in range(10):
    
    # read data
    X_train = pd.read_csv("../../result/input_perturb_go/%d/X_train_stratified" % rs, sep="\t", index_col=0).values
    X_valid = pd.read_csv("../../result/input_perturb_go/%d/X_valid_stratified" % rs, sep="\t", index_col=0).values
    X_test = pd.read_csv("../../result/input_perturb_go/%d/X_test_stratified" % rs, sep="\t", index_col=0).values
    Y_train = pd.read_csv("../../result/input_perturb_go/%d/Y_train_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_valid = pd.read_csv("../../result/input_perturb_go/%d/Y_valid_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test = pd.read_csv("../../result/input_perturb_go/%d/Y_test_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test_gene = pd.read_csv("../../result/input_perturb_go/%d/Y_test_stratified" % rs, sep="\t", index_col=0).index

    # concat all subsets
    X = np.concatenate([X_train, X_valid, X_test])
    Y = np.concatenate([Y_train, Y_valid, Y_test])
    train_mask = np.concatenate([[True] * len(X_train), [False] * len(X_valid), [False] * len(X_test)])
    valid_mask = np.concatenate([[False] * len(X_train), [True] * len(X_valid), [False] * len(X_test)])
    test_mask = np.concatenate([[False] * len(X_train), [False] * len(X_valid), [True] * len(X_test)])
    
    # mask of whether a test node is in the DAGMA graph
    dag = pd.read_csv("../../result/network_perturb_go/DAGMA_thresholdAdaptive.tsv", sep="\t", header=None)
    id2genes = pd.read_csv("../../result/network_perturb_go/valid_genes", sep="\t").set_index("ID")['genes'].to_dict()
    dag[0] = dag[0].map(id2genes)
    dag[1] = dag[1].map(id2genes)
    dag_genes = list(set.union(set(dag[0]), set(dag[1])))
    test_inDAG = Y_test_gene.isin(dag_genes)

    for network_name in ["DAGMA_thresholdAdaptive", "STRING", "BIOGRID", "Combine", "ChIP_hTFtarget", "ChIP_TIP", "ChIP_TIP_K562", "CoExpr_ENCODE_K562_0.75", "CoExpr_GTEx_WholeBlood_0.75", "CoExpr_perturb_0.5", "Random_ER", "Random_SF"]:

        # read network
        edge_index = torch.tensor(pd.read_csv("../../result/network_perturb_go/%s.tsv" % network_name, sep="\t", header=None).values.T)

        # convert to data object
        data = torch_geometric.data.Data(x=torch.tensor(X).float(), edge_index=edge_index, y=torch.tensor(Y).float())
        data.train_mask = torch.tensor(train_mask)
        data.valid_mask = torch.tensor(valid_mask)
        data.test_mask = torch.tensor(test_mask)
        data.test_net_mask = np.concatenate([[False] * len(Y_train), [False] * len(Y_valid), test_inDAG])

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data = data.to(device)


        # repeat the whole process multiple times for mean and std of the loss
        test_loss_reps = []
        test_net_loss_reps = []

        os.makedirs("../../result/model_perturb_go/%s/%d/" % (network_name, rs), exist_ok=True)

        for rep in range(10):
            # tune hyperparameters
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=20, timeout=600)
            best_params = study.best_params

            # final model with best hyperparameters and use train and valid data to train
            final_model = GCN(best_params["size1"], best_params["size2"], best_params["size3"]).to(device)
            optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])

            # train final model
            for epoch in range(1, best_params["num_epochs"] + 1):
                train_loss = train(final_model, data, optimizer, data.train_mask|data.valid_mask)

            # prediction from final model
            final_model.eval()
            with torch.no_grad():
                out, X1 = final_model(data)
                # loss on the testing set
                loss1 = F.mse_loss(out[data.test_mask].view(-1), data.y[data.test_mask])
                test_loss_reps.append(loss1.item())
                # loss on the testing set, but only the nodes in the DAGMA
                loss2 = F.mse_loss(out[data.test_net_mask].view(-1), data.y[data.test_net_mask])
                test_net_loss_reps.append(loss2.item())

            # save model, hyperparameters and results
            prefix = "../../result/model_perturb_go/%s/%d/model%d" % (network_name, rs, rep)
            torch.save(final_model.state_dict(), prefix+".model")
            with open(prefix+".para", "wb") as f:
                pickle.dump([best_params, X1.cpu().numpy(), out.cpu().numpy(), loss1.item(), loss2.item()], f)

[I 2023-12-17 15:08:20,405] A new study created in memory with name: no-name-63f84a90-22ea-4d95-aa05-439240de07e6
[I 2023-12-17 15:08:26,502] Trial 0 finished with value: 352.422607421875 and parameters: {'size1': 211, 'size2': 279, 'size3': 91, 'num_epochs': 124, 'lr': 0.0017824874371330953}. Best is trial 0 with value: 352.422607421875.
[I 2023-12-17 15:08:27,009] Trial 1 finished with value: 336.494140625 and parameters: {'size1': 493, 'size2': 364, 'size3': 269, 'num_epochs': 128, 'lr': 0.00937272066994294}. Best is trial 1 with value: 336.494140625.
[I 2023-12-17 15:08:27,772] Trial 2 finished with value: 354.76873779296875 and parameters: {'size1': 632, 'size2': 23, 'size3': 24, 'num_epochs': 189, 'lr': 0.0025342921839831864}. Best is trial 1 with value: 336.494140625.
[I 2023-12-17 15:08:28,285] Trial 3 finished with value: 340.2245788574219 and parameters: {'size1': 884, 'size2': 220, 'size3': 391, 'num_epochs': 104, 'lr': 0.008440965683060395}. Best is trial 1 with value: 336.

[I 2023-12-17 15:08:38,875] Trial 10 pruned. 
[I 2023-12-17 15:08:38,900] Trial 11 pruned. 
[I 2023-12-17 15:08:38,929] Trial 12 pruned. 
[I 2023-12-17 15:08:38,959] Trial 13 pruned. 
[I 2023-12-17 15:08:39,012] Trial 14 pruned. 
[I 2023-12-17 15:08:39,055] Trial 15 finished with value: 370.33099365234375 and parameters: {'size1': 451, 'size2': 342, 'size3': 182, 'num_epochs': 3, 'lr': 0.007102332763240414}. Best is trial 2 with value: 296.2476501464844.
[I 2023-12-17 15:08:39,113] Trial 16 pruned. 
[I 2023-12-17 15:08:39,145] Trial 17 pruned. 
[I 2023-12-17 15:08:39,178] Trial 18 pruned. 
[I 2023-12-17 15:08:39,237] Trial 19 pruned. 
[I 2023-12-17 15:08:39,420] A new study created in memory with name: no-name-05d62f70-26f7-461d-983b-759e1e239246
[I 2023-12-17 15:08:40,088] Trial 0 finished with value: 371.2703552246094 and parameters: {'size1': 267, 'size2': 174, 'size3': 58, 'num_epochs': 185, 'lr': 0.00900289182561413}. Best is trial 0 with value: 371.2703552246094.
[I 2023-12-17 15

[I 2023-12-17 15:08:50,891] Trial 8 pruned. 
[I 2023-12-17 15:08:51,497] Trial 9 finished with value: 351.5282897949219 and parameters: {'size1': 840, 'size2': 71, 'size3': 144, 'num_epochs': 125, 'lr': 0.007262989904769762}. Best is trial 0 with value: 255.88922119140625.
[I 2023-12-17 15:08:51,531] Trial 10 pruned. 
[I 2023-12-17 15:08:51,598] Trial 11 pruned. 
[I 2023-12-17 15:08:51,708] Trial 12 pruned. 
[I 2023-12-17 15:08:51,741] Trial 13 pruned. 
[I 2023-12-17 15:08:51,780] Trial 14 pruned. 
[I 2023-12-17 15:08:51,820] Trial 15 pruned. 
[I 2023-12-17 15:08:51,892] Trial 16 pruned. 
[I 2023-12-17 15:08:51,939] Trial 17 pruned. 
[I 2023-12-17 15:08:51,977] Trial 18 pruned. 
[I 2023-12-17 15:08:52,781] Trial 19 finished with value: 351.8830261230469 and parameters: {'size1': 742, 'size2': 345, 'size3': 347, 'num_epochs': 166, 'lr': 0.0044199746853399394}. Best is trial 0 with value: 255.88922119140625.
[I 2023-12-17 15:08:53,115] A new study created in memory with name: no-name-12e

[I 2023-12-17 15:09:10,128] Trial 2 finished with value: 623.501953125 and parameters: {'size1': 100, 'size2': 325, 'size3': 87, 'num_epochs': 13, 'lr': 0.000825610721472108}. Best is trial 1 with value: 307.1733703613281.
[I 2023-12-17 15:09:10,179] Trial 3 finished with value: 394.83203125 and parameters: {'size1': 655, 'size2': 149, 'size3': 463, 'num_epochs': 3, 'lr': 0.007750421616003353}. Best is trial 1 with value: 307.1733703613281.
[I 2023-12-17 15:09:10,245] Trial 4 finished with value: 469.8289794921875 and parameters: {'size1': 1005, 'size2': 63, 'size3': 244, 'num_epochs': 3, 'lr': 0.003770565180454445}. Best is trial 1 with value: 307.1733703613281.
[I 2023-12-17 15:09:10,256] Trial 5 pruned. 
[I 2023-12-17 15:09:10,736] Trial 6 finished with value: 331.27587890625 and parameters: {'size1': 294, 'size2': 43, 'size3': 414, 'num_epochs': 74, 'lr': 0.008839619463365114}. Best is trial 1 with value: 307.1733703613281.
[I 2023-12-17 15:09:10,745] Trial 7 pruned. 
[I 2023-12-17

[I 2023-12-17 15:09:38,013] Trial 1 finished with value: 451.57080078125 and parameters: {'size1': 79, 'size2': 448, 'size3': 333, 'num_epochs': 195, 'lr': 0.008734205779792137}. Best is trial 0 with value: 352.3435363769531.
[I 2023-12-17 15:09:41,439] Trial 2 finished with value: 446.7466735839844 and parameters: {'size1': 985, 'size2': 440, 'size3': 422, 'num_epochs': 185, 'lr': 0.009973280899432377}. Best is trial 0 with value: 352.3435363769531.
[I 2023-12-17 15:09:41,669] Trial 3 finished with value: 337.5172119140625 and parameters: {'size1': 146, 'size2': 228, 'size3': 53, 'num_epochs': 36, 'lr': 0.009151818692915138}. Best is trial 3 with value: 337.5172119140625.
[I 2023-12-17 15:09:43,536] Trial 4 finished with value: 409.2725830078125 and parameters: {'size1': 788, 'size2': 435, 'size3': 331, 'num_epochs': 115, 'lr': 0.007163106318790347}. Best is trial 3 with value: 337.5172119140625.
[I 2023-12-17 15:09:43,949] Trial 5 pruned. 
[I 2023-12-17 15:09:43,989] Trial 6 pruned. 

...

[I 2023-12-17 19:02:38,725] Trial 5 pruned. 
[I 2023-12-17 19:02:38,768] Trial 6 pruned. 
[I 2023-12-17 19:02:38,818] Trial 7 pruned. 
[I 2023-12-17 19:02:39,502] Trial 8 finished with value: 586.1640014648438 and parameters: {'size1': 856, 'size2': 170, 'size3': 379, 'num_epochs': 100, 'lr': 0.00469650116853704}. Best is trial 2 with value: 511.5299072265625.
[I 2023-12-17 19:02:39,538] Trial 9 pruned. 
[I 2023-12-17 19:02:39,585] Trial 10 pruned. 
[I 2023-12-17 19:02:39,632] Trial 11 pruned. 
[I 2023-12-17 19:02:39,701] Trial 12 pruned. 
[I 2023-12-17 19:02:39,816] Trial 13 pruned. 
[I 2023-12-17 19:02:39,858] Trial 14 pruned. 
[I 2023-12-17 19:02:39,903] Trial 15 pruned. 
[I 2023-12-17 19:02:39,951] Trial 16 pruned. 
[I 2023-12-17 19:02:40,070] Trial 17 pruned. 
[I 2023-12-17 19:02:40,096] Trial 18 pruned. 
[I 2023-12-17 19:02:40,208] Trial 19 pruned. 
[I 2023-12-17 19:02:40,446] A new study created in memory with name: no-name-63c9fe1d-7400-4e5b-bbb3-baae7d732ec8
[I 2023-12-17 19:0

[I 2023-12-17 19:02:58,043] Trial 3 finished with value: 538.2275390625 and parameters: {'size1': 182, 'size2': 80, 'size3': 44, 'num_epochs': 162, 'lr': 0.008012456888853724}. Best is trial 0 with value: 516.360107421875.
[I 2023-12-17 19:02:58,752] Trial 4 finished with value: 507.82611083984375 and parameters: {'size1': 702, 'size2': 359, 'size3': 73, 'num_epochs': 113, 'lr': 0.008904670432755838}. Best is trial 4 with value: 507.82611083984375.
[I 2023-12-17 19:02:58,761] Trial 5 pruned. 
[I 2023-12-17 19:02:58,892] Trial 6 pruned. 
[I 2023-12-17 19:02:59,230] Trial 7 pruned. 
[I 2023-12-17 19:02:59,248] Trial 8 pruned. 
[I 2023-12-17 19:02:59,596] Trial 9 pruned. 
[I 2023-12-17 19:02:59,640] Trial 10 pruned. 
[I 2023-12-17 19:02:59,675] Trial 11 pruned. 
[I 2023-12-17 19:02:59,721] Trial 12 pruned. 
[I 2023-12-17 19:02:59,756] Trial 13 pruned. 
[I 2023-12-17 19:02:59,804] Trial 14 pruned. 
[I 2023-12-17 19:02:59,830] Trial 15 pruned. 
[I 2023-12-17 19:03:00,075] Trial 16 finished 

[I 2023-12-17 19:03:15,158] Trial 16 pruned. 
[I 2023-12-17 19:03:15,222] Trial 17 pruned. 
[I 2023-12-17 19:03:15,269] Trial 18 pruned. 
[I 2023-12-17 19:03:15,311] Trial 19 pruned. 
[I 2023-12-17 19:03:15,448] A new study created in memory with name: no-name-38ee242c-cdc1-4516-aea5-112b1b4b7ec2
[I 2023-12-17 19:03:16,299] Trial 0 finished with value: 669.2003173828125 and parameters: {'size1': 573, 'size2': 310, 'size3': 164, 'num_epochs': 159, 'lr': 0.007604318537222744}. Best is trial 0 with value: 669.2003173828125.
[I 2023-12-17 19:03:16,999] Trial 1 finished with value: 639.435302734375 and parameters: {'size1': 757, 'size2': 172, 'size3': 433, 'num_epochs': 118, 'lr': 0.0036213173728269275}. Best is trial 1 with value: 639.435302734375.
[I 2023-12-17 19:03:17,100] Trial 2 finished with value: 715.6492919921875 and parameters: {'size1': 494, 'size2': 370, 'size3': 35, 'num_epochs': 17, 'lr': 0.005388981707694834}. Best is trial 1 with value: 639.435302734375.
[I 2023-12-17 19:03

[I 2023-12-17 19:03:28,958] Trial 10 pruned. 
[I 2023-12-17 19:03:28,992] Trial 11 pruned. 
[I 2023-12-17 19:03:29,034] Trial 12 pruned. 
[I 2023-12-17 19:03:29,059] Trial 13 pruned. 
[I 2023-12-17 19:03:29,103] Trial 14 pruned. 
[I 2023-12-17 19:03:29,146] Trial 15 pruned. 
[I 2023-12-17 19:03:29,243] Trial 16 pruned. 
[I 2023-12-17 19:03:29,272] Trial 17 pruned. 
[I 2023-12-17 19:03:29,312] Trial 18 pruned. 
[I 2023-12-17 19:03:29,360] Trial 19 pruned. 
[I 2023-12-17 19:03:29,463] A new study created in memory with name: no-name-b418e658-9c5f-4072-94b1-e582ad03a4a4
[I 2023-12-17 19:03:29,561] Trial 0 finished with value: 606.4435424804688 and parameters: {'size1': 626, 'size2': 448, 'size3': 506, 'num_epochs': 13, 'lr': 0.0018544757231277605}. Best is trial 0 with value: 606.4435424804688.
[I 2023-12-17 19:03:30,107] Trial 1 finished with value: 614.2999267578125 and parameters: {'size1': 190, 'size2': 315, 'size3': 182, 'num_epochs': 140, 'lr': 0.0014993270544116045}. Best is trial 

[I 2023-12-17 19:03:45,570] Trial 0 finished with value: 635.3639526367188 and parameters: {'size1': 864, 'size2': 124, 'size3': 124, 'num_epochs': 128, 'lr': 0.00580871256011838}. Best is trial 0 with value: 635.3639526367188.
[I 2023-12-17 19:03:46,690] Trial 1 finished with value: 628.6320190429688 and parameters: {'size1': 811, 'size2': 331, 'size3': 464, 'num_epochs': 168, 'lr': 0.007640545105231855}. Best is trial 1 with value: 628.6320190429688.
[I 2023-12-17 19:03:47,240] Trial 2 finished with value: 558.9207153320312 and parameters: {'size1': 530, 'size2': 422, 'size3': 217, 'num_epochs': 99, 'lr': 0.000194920882316086}. Best is trial 2 with value: 558.9207153320312.
[I 2023-12-17 19:03:47,691] Trial 3 finished with value: 609.7247314453125 and parameters: {'size1': 45, 'size2': 383, 'size3': 426, 'num_epochs': 114, 'lr': 0.0045203155421683966}. Best is trial 2 with value: 558.9207153320312.
[I 2023-12-17 19:03:48,274] Trial 4 finished with value: 600.2157592773438 and paramet